<a href="https://colab.research.google.com/github/MohuaSinha/EVA4/blob/master/S15A/S15A_DenseDepthMap_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


<a href="https://colab.research.google.com/github/ialhashim/DenseDepth/blob/master/DenseDepth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/mohuasinha/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 251 (delta 6), reused 12 (delta 4), pack-reused 237
Receiving objects: 100% (251/251), 11.80 MiB | 13.58 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [3]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-05-10 15:16:51--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.102.3
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.102.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  78.4MB/s    in 2.1s    

2020-05-10 15:16:53 (78.4 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



In [0]:
#Manually make a copy of fg_bg.zip as depth_fg_bg.zip
#cp '/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/fg_bg.zip' '/content/depth_fg_bg.zip'

In [0]:
import zipfile

archive = zipfile.ZipFile('/content/gdrive/My Drive/Colab Notebooks/EVA4/S14_15/output/fg_bg.zip')
for file in archive.namelist():
    if file.startswith('image/'):
        archive.extract(file, '/content/')

In [0]:
#!cp '/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/fg_bg.zip' '/content/'

In [7]:
!ls '/content/image' | wc -l

400000


In [0]:
!mkdir '/content/depthmap'

In [9]:
!ls '/content/depthmap' | wc -l

0


In [0]:
#!rm -rf '/content/depthmap'

In [11]:
!pwd

/content


In [12]:
cd DenseDepth

/content/DenseDepth


In [0]:
import sys
workingdir = '/content/DenseDepth'
sys.path.append(workingdir)

In [14]:
!ls

augment.py	demo.py		  examples   model.py	Tensorflow
callbacks.py	demo_rgb.npy	  layers.py  nyu.h5	test.py
data.py		DenseDepth.ipynb  LICENSE    PyTorch	train.py
demo_depth.npy	evaluate.py	  loss.py    README.md	utils.py


In [0]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import notebook
import zipfile

def load_images(path, start, end):
    loaded_images = []
    for i in range(start, end+1):
        x = np.clip(np.asarray(Image.open(f"{path}fg_bg_{str(i)}.jpg" ).resize((256, 256)), dtype=float) / 255, 0, 1)
        #x = np.clip(np.asarray(Image.open(f"{path}fg_bg_{str(i)}.jpg" ), dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)


def write_images(out_zip, path, start, outputs, inputs=None, gt=None, is_colormap=True, is_rescale=True, log=False):
    import matplotlib.pyplot as plt
    import skimage
    from skimage.transform import resize

    plasma = plt.get_cmap('gray')

    shape = (outputs[0].shape[0], outputs[0].shape[1], 3)
    if log:
        print(shape)
    
    #all_images = []
    start = start
    for i in notebook.tqdm(range(outputs.shape[0]),desc = f"Loading from {start} images") :
    
        if is_colormap:
            rescaled = outputs[i][:,:,0]
            if is_rescale:
                rescaled = rescaled - np.min(rescaled)
                rescaled = rescaled / np.max(rescaled)
  
            plt.figure(figsize=(2.24,2.24),dpi=100)
            matplotlib_image = plt.imshow(plasma(rescaled)[:,:,:3])
            pil_image = Image.fromarray(np.uint8( ( matplotlib_image.get_array()*255))).convert("L").resize((220, 220))
            #pil_image = Image.fromarray(np.uint8( ( matplotlib_image.get_array()*255))).convert("L")
            pil_image.save('imgtemp.jpg', optimize=True, quality=30)
            out_zip.write('imgtemp.jpg',f'depthmap/depth_{str(start)}.jpg')
            #if log:
            #    print("Saving Image: ", f"{path}depth_{str(start)}.jpg")
            pil_image.save(f"{path}depth_{str(start)}.jpg")
            plt.close()
            pil_image = None 
            matplotlib_image = None
            start+=1

In [0]:
#!ls '/content/image'

In [1]:
import test
import time
import gc


INPATH = "/content/image/"
OUTPATH = "/content/depthmap/"

start_time = time.time()

total_images = 400000
#total_images = 4000
batch_size = 500
num_iter = int(total_images/batch_size)
#num_iter = 250
image_start = 200000

print("Generating Depth Images for:", total_images, "in batches of:", batch_size, "with ", num_iter, "iterations")

model = test.loadmodel()

outputs = []
inputs = []
log = False

for  i in range(1, num_iter+1):
  iter_start = time.time()
  out_zip =zipfile.ZipFile(f'/content/gdrive/My Drive/Colab Notebooks/EVA4/S14_15/output/depth_map_from 200000.zip', mode='a', compression=zipfile.ZIP_STORED)
  start = image_start
  end = image_start + batch_size - 1
  print("Generating Image: ", image_start, " to ", end)
  inputs = load_images(INPATH, start, end)
  print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))
  outputs = test.predict_output(model, inputs, log=True)
  write_images(out_zip, OUTPATH, start, outputs.copy(), log=True)
  print("Iteration:", i, "--- %s seconds ---" % (time.time() - iter_start), " --- Total : %s seconds ---" % (time.time() - start_time))
  image_start = image_start + batch_size
  inputs = None
  outputs = None
  out_zip.close()
  gc.collect()

print("--- %s seconds ---" % (time.time() - start_time))


Generating Depth Images for: 400000 in batches of: 500 with  800 iterations


AttributeError: ignored

In [0]:
!ls '/content/depthmap' | wc -l

In [0]:
!zip depthmap_121001_to_151000.zip '/content/depthmap'

In [0]:
!zip -r depthmap_121001_to_151500.zip '/content/depthmap'

In [0]:
cp '/content/depthmap_121001_to_151500.zip' '/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/'

In [0]:
!mkdir '/content/depthmap'

In [0]:
!ls '/content/depthmap' | wc -l

In [0]:
#!rm -rf '/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/depthmap/'

In [0]:
!mkdir '/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/depthmap/'

In [0]:
!cp -r '/content/depthmap/' '/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/'

In [0]:
!ls '/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/depthmap' | wc -l